<a href="https://colab.research.google.com/github/lkarjun/malayalam-language-model/blob/main/Malayalam-Language-Model/malayalam-language-model_awd_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Malayalam Language Model AWD-LSTM

### Imports

In [1]:
!pip install -qq fastai==2.5.3 sentencepiece transformers datasets

     |████████████████████████████████| 189 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 54.0 MB/s 
     |████████████████████████████████| 3.5 MB 50.4 MB/s 
     |████████████████████████████████| 311 kB 27.6 MB/s 
     |████████████████████████████████| 56 kB 3.3 MB/s 
     |████████████████████████████████| 596 kB 54.0 MB/s 
     |████████████████████████████████| 6.8 MB 41.9 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 895 kB 56.5 MB/s 
     |████████████████████████████████| 133 kB 57.1 MB/s 
     |████████████████████████████████| 1.1 MB 38.6 MB/s 
     |████████████████████████████████| 243 kB 43.7 MB/s 
     |████████████████████████████████| 271 kB 45.0 MB/s 
     |████████████████████████████████| 144 kB 45.6 MB/s 
     |████████████████████████████████| 94 kB 2.0 MB/s 


In [5]:
from fastai.text.all import *
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset

In [6]:
dset = load_dataset("lkarjun/Malayalam-Articles")

Using custom data configuration lkarjun--Malayalam-Articles-d44c52244000c266


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/252M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/81.3M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/lkarjun--Malayalam-Articles-d44c52244000c266/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

### Load datasets

In [9]:
train = dset['train'].to_pandas()[:4000]

In [10]:
train.isna().sum()

content     12
is_valid     0
dtype: int64

In [11]:
train.dropna(inplace = True)

### Tokenizer

In [12]:
pre_tokenizer = PreTrainedTokenizerFast.from_pretrained("lkarjun/malayalam-language-model")

In [13]:
def tokenize(text):
  assert type(text) == str, "check the input type"
  toks = pre_tokenizer.tokenize(text)
  toks = ['[CLS]'] + toks + ['[SEP]']
  toks = pre_tokenizer.convert_tokens_to_ids(toks)
  return tensor(toks)

In [14]:
class MlTokenizer(Transform):
    def __init__(self, tokenizer): 
      self.tokenizer = tokenizer
    def encodes(self, x): 
        return x if isinstance(x, Tensor) else tokenize(x)
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [15]:
tokenizer = MlTokenizer(pre_tokenizer)
vocab = pre_tokenizer.vocab

### Dataloder

In [21]:
tokenized = [tokenize(t) for t in progress_bar(train['content'])]

In [18]:
splits = RandomSplitter(.2, 7)(train)
splits

((#3191) [3272,1806,1347,2860,2784,2521,1463,1081,2698,2000...],
 (#797) [3787,1580,2931,34,3335,3649,1161,1071,696,2661...])

In [22]:
tfms = TfmdLists(tokenized, 
                 tokenizer, 
                 splits=splits, 
                 dl_type=LMDataLoader)

In [28]:
mldls = tfms.dataloaders(bs=6, seq_len=72)

In [29]:
mldls.vocab = vocab

In [30]:
mldls.show_batch(max_n = 4)

,text,text_
0,[CLS] ഉക്ര ##േന ##ിലെ റോസ് ##‍ ##റ്റോ ##ക്സ് മേഖലയിൽ സ്ഥിതിചെയ്യുന്ന പടിഞ്ഞാറൻ ഉക്ര ##ൈനിലെ ഏറ്റവും രസകരമായ ഫിസി ##യി ##യോഗ ##്ര ##ാഫിക് പ്രദേശങ്ങളിൽ ഒന്നാണ് യാ ##വോ ##റിവ് ##സ്കി നാഷണൽ പാർക്ക് [SEP] [CLS] 1970 - കളിൽ ലോകമെങ്ങും വൃക്ഷ സ്നേഹ ##ത്തിൻറെ പുതിയ മാതൃക കാണിച്ചു കൊടുത്ത ചിപ് ##കോ പ്രസ്ഥാനത്തിന് പുതു ##ജീവൻ നൽകി ഉത്തരാഖണ്ഡിലെ വനിതകൾ. ബാഗ ##േശ്വർ ജില്ലയില ജാ ##ഖ് ##നി ഗ്രാമത്തിലുള്ള വനിത ##കളാണ് വനനശ ##ീകരണ ##ത്തിനെതിരെ മരങ്ങളിൽ കെട്ടിപ്പിടിച്ചു ##ള്ള സമര ##മുറ ##യുമായി രംഗത്തെത്തിയത്. പുതിയ റോഡ്,ഉക്ര ##േന ##ിലെ റോസ് ##‍ ##റ്റോ ##ക്സ് മേഖലയിൽ സ്ഥിതിചെയ്യുന്ന പടിഞ്ഞാറൻ ഉക്ര ##ൈനിലെ ഏറ്റവും രസകരമായ ഫിസി ##യി ##യോഗ ##്ര ##ാഫിക് പ്രദേശങ്ങളിൽ ഒന്നാണ് യാ ##വോ ##റിവ് ##സ്കി നാഷണൽ പാർക്ക് [SEP] [CLS] 1970 - കളിൽ ലോകമെങ്ങും വൃക്ഷ സ്നേഹ ##ത്തിൻറെ പുതിയ മാതൃക കാണിച്ചു കൊടുത്ത ചിപ് ##കോ പ്രസ്ഥാനത്തിന് പുതു ##ജീവൻ നൽകി ഉത്തരാഖണ്ഡിലെ വനിതകൾ. ബാഗ ##േശ്വർ ജില്ലയില ജാ ##ഖ് ##നി ഗ്രാമത്തിലുള്ള വനിത ##കളാണ് വനനശ ##ീകരണ ##ത്തിനെതിരെ മരങ്ങളിൽ കെട്ടിപ്പിടിച്ചു ##ള്ള സമര ##മുറ ##യുമായി രംഗത്തെത്തിയത്. പുതിയ റോഡ് നിർമ്മിക്കുന്ന
1,. തി രുച ##് ##ചിറ ##പ്പള്ളിയിൽ നിന്നും പിന്നീട് ചെന്ന ##െയിലേക്ക് മാറിയ സരസ്വതി ##യുടെ കഥ വായിക്കാന ##ിടയ ##ായ അന്നത്തെ മുഖ്യമന്ത്രി ജയലളിത മുൻകൈ ##യെടുത്തു കൊണ്ട് താമസിക്കാന ##ൊരു വീടും പെൻഷൻ തുകയും അനുവദിക്കുന്നത് പല കഷ്ട ##തകളും സഹിച്ചു കൊണ്ടായിരുന്നു സരസ്വതി ജീവിച്ചിരുന്നത് എന്നത് വേദനിപ്പിക്കുന്ന വാർത്തയായിരുന്നു. തന്റെ തുച്ഛമായ പെൻഷൻ തുകയ ##ിൽ നിന്നും നല്ലൊരു ശതമാനം 2004 ലെ സുനാമി ബാധിത ##ർക്ക് നൽകിയ വാർത്ത പുറത്തു വന്നപ്പോഴാണ് പിന്നീട് ആ ധീര വനിത ##യെക്കുറിച്ചു ഇന്നത്തെ തലമുറ കൂടുതൽ മനസിലാക്കിയത്. 2018 ൽ മരണമടയ ##ുമ്പോൾ ഇന്ത്യയുടെ,തി രുച ##് ##ചിറ ##പ്പള്ളിയിൽ നിന്നും പിന്നീട് ചെന്ന ##െയിലേക്ക് മാറിയ സരസ്വതി ##യുടെ കഥ വായിക്കാന ##ിടയ ##ായ അന്നത്തെ മുഖ്യമന്ത്രി ജയലളിത മുൻകൈ ##യെടുത്തു കൊണ്ട് താമസിക്കാന ##ൊരു വീടും പെൻഷൻ തുകയും അനുവദിക്കുന്നത് പല കഷ്ട ##തകളും സഹിച്ചു കൊണ്ടായിരുന്നു സരസ്വതി ജീവിച്ചിരുന്നത് എന്നത് വേദനിപ്പിക്കുന്ന വാർത്തയായിരുന്നു. തന്റെ തുച്ഛമായ പെൻഷൻ തുകയ ##ിൽ നിന്നും നല്ലൊരു ശതമാനം 2004 ലെ സുനാമി ബാധിത ##ർക്ക് നൽകിയ വാർത്ത പുറത്തു വന്നപ്പോഴാണ് പിന്നീട് ആ ധീര വനിത ##യെക്കുറിച്ചു ഇന്നത്തെ തലമുറ കൂടുതൽ മനസിലാക്കിയത്. 2018 ൽ മരണമടയ ##ുമ്പോൾ ഇന്ത്യയുടെ ആദ്യ
2,"##ട്ട ##ത്തിന്റെയും കു ##ച്ചിപ്പ ##ുടിയ ##ുടെയും ചില സവിശേഷതകൾ സ്വി ##കരി ##ച്ചുകൊണ്ട് ക്ലാസിക്കൽ ശൈലിയിൽ രൂപമെടുത്ത ##താണ് ഈ നൃത്ത ##വിശേഷം എന്നും കരുതപ്പെടുന്നു. ‘ ത്രി ##ഭംഗ ’ ഒഡീസ്സി ##യുമായി ബന്ധപ്പെട്ടിരിക്കുന്നു. സംയുക്ത പാണ ##ി ##ഗ്രാഹ ##ി, സോണ ##ാൽ മാൻ ##സിങ്ങ്, മാധവി മു ##ദ്‌ ##ഗൽ, കിരൺ സൈഗ ##ാൾ, റാണി കരൺ എന്നിവർ പ്രശസ്തരായ ഒഡീസ്സി നർത്തക ##രാണ്. ജയ ##ദേവ ##രുടെ ‘ ഗീതാ ##ഗോവിന്ദ ##ത്തിലെ ’ കവിതകളാണ് ഒഡീസ്സി നൃത്തത്തിന്റെ സംഗീത ##ത്തിനായി പ്രധാനമായും ഉപയോഗിച്ചിരിക്കുന്നത്","##ത്തിന്റെയും കു ##ച്ചിപ്പ ##ുടിയ ##ുടെയും ചില സവിശേഷതകൾ സ്വി ##കരി ##ച്ചുകൊണ്ട് ക്ലാസിക്കൽ ശൈലിയിൽ രൂപമെടുത്ത ##താണ് ഈ നൃത്ത ##വിശേഷം എന്നും കരുതപ്പെടുന്നു. ‘ ത്രി ##ഭംഗ ’ ഒഡീസ്സി ##യുമായി ബന്ധപ്പെട്ടിരിക്കുന്നു. സംയുക്ത പാണ ##ി ##ഗ്രാഹ ##ി, സോണ ##ാൽ മാൻ ##സിങ്ങ്, മാധവി മു ##ദ്‌ ##ഗൽ, കിരൺ സൈഗ ##ാൾ, റാണി കരൺ എന്നിവർ പ്രശസ്തരായ ഒഡീസ്സി നർത്തക ##രാണ്. ജയ ##ദേവ ##രുടെ ‘ ഗീതാ ##ഗോവിന്ദ ##ത്തിലെ ’ കവിതകളാണ് ഒഡീസ്സി നൃത്തത്തിന്റെ സംഗീത ##ത്തിനായി പ്രധാനമായും ഉപയോഗിച്ചിരിക്കുന്നത്."
3,". അവിടെ ചെന്നകാലത്ത്, ഫിഗറേറ്റീവ് ചിത്രങ്ങളാണ് ഞാന്‍ ചെയ്തുകൊണ്ടിരുന്നത്. ഒരുതരം ക്യൂബി ##സ്റ്റിക് ടൈപ്പ്. ധാരാളം ഡ്രോയി ##ങ്ങുകള്‍ - ബ്ലാക്ക് ല്ക്ക വൈറ്റ് - ഉണ്ടായിരുന്നു. പൊതുവേ പറഞ്ഞാല്‍, ജര്‍മന്‍ ഇംപ്രഷനി ##സത്തില്‍ പെടുത്താവുന്ന തരം പെയിന്റിങ്ങുകള്‍. 68 - ല്‍ അതു നിര്‍ത്തി. പിന്നെ ചെറിയ പെയിന്റിങ്ങുകള്‍ ചെയ്തുതുടങ്ങി. ആയിടെ അജിത് മുഖര്‍ജിയുടെ ഒരു പുസ്തകമിറങ്ങി - താന്ത്രിക് ആര്‍ട്ട്. അത്തരം ചിത്രങ്ങള്‍ ഞാനും ചെയ്തുതുടങ്ങി. വിശ്വനാഥനും ഏതാണ്ടി ##ങ്ങനെയാണ് ചെയ്തത്. പക്ഷേ, അതു കുറേക്കൂടി വലുതായിരുന്നു. അക്കാലത്ത് കെ","അവിടെ ചെന്നകാലത്ത്, ഫിഗറേറ്റീവ് ചിത്രങ്ങളാണ് ഞാന്‍ ചെയ്തുകൊണ്ടിരുന്നത്. ഒരുതരം ക്യൂബി ##സ്റ്റിക് ടൈപ്പ്. ധാരാളം ഡ്രോയി ##ങ്ങുകള്‍ - ബ്ലാക്ക് ല്ക്ക വൈറ്റ് - ഉണ്ടായിരുന്നു. പൊതുവേ പറഞ്ഞാല്‍, ജര്‍മന്‍ ഇംപ്രഷനി ##സത്തില്‍ പെടുത്താവുന്ന തരം പെയിന്റിങ്ങുകള്‍. 68 - ല്‍ അതു നിര്‍ത്തി. പിന്നെ ചെറിയ പെയിന്റിങ്ങുകള്‍ ചെയ്തുതുടങ്ങി. ആയിടെ അജിത് മുഖര്‍ജിയുടെ ഒരു പുസ്തകമിറ

### Traning Language Model

In [42]:
x, y = mldls.one_batch()
x.shape, y.shape

(torch.Size([6, 72]), torch.Size([6, 72]))

In [31]:
doc(language_model_learner)

language_model_learner(dls, arch, config=None, drop_mult=1.0, backwards=False, pretrained=True, pretrained_fnames=None, loss_func=None, opt_func=<function Adam at 0x7fb239826cb0>, lr=0.001, splitter=<function trainable_params at 0x7fb247d77dd0>, cbs=None, metrics=None, path=None, model_dir='models', wd=None, wd_bn_bias=False, train_bn=True, moms=(0.95, 0.85, 0.95))
Create a `Learner` with a language model from `dls` and `arch`.

To get a prettier result with hyperlinks to source code and documentation, install nbdev: pip install nbdev


In [43]:
learn = language_model_learner(
            mldls, 
            AWD_LSTM,
            drop_mult=.3,
            pretrained = False,
            metrics = Perplexity()
            ).to_fp16()

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
learn.fit(1, 3e-4)